In [9]:
from src.data.data import *
from src.embedor import *
from src.plotting import *
import pandas as pd
import matplotlib
import seaborn as sns
import argparse
import umap
import numpy as np
from sklearn.manifold import TSNE, Isomap, SpectralEmbedding
import phate
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def est_geodesic_distance(X, k=15, scale_factor=1):
    """
    Estimate the geodesic distance using the knn graph.
    :param k: number of neighbors
    :return: geodesic distance matrix. Note: infinite values get snapped to a scaled version of the max finite value
    """
    from sklearn.neighbors import kneighbors_graph
    from scipy.sparse.csgraph import shortest_path
    knn = kneighbors_graph(X, n_neighbors=k, mode='distance', include_self=False)
    knn = knn.toarray()
    knn[knn == 0] = np.inf
    # knn[knn > 0] = 1
    dists = shortest_path(knn, method='D', directed=False)
    # max finite value
    max_dist = np.nanmax(dists[~np.isinf(dists)])
    dists[np.isinf(dists)] = max_dist * scale_factor
    return dists

def geodesic_dist_experiment(data_dict):
    gt_geodesic_distance = est_geodesic_distance(data_dict['noiseless_data'], k=15)
    embedor = EmbedOR()
    embedding = embedor.fit_transform(data_dict['data'])
    # node_indices = embedor.G.nodes
    umap_emb = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='euclidean').fit_transform(data_dict['data'])
    tsne_emb = TSNE().fit_transform(data_dict['data']) 
    phate_emb = phate.PHATE().fit_transform(data_dict['data'])
    isomap_emb = Isomap().fit_transform(data_dict['data'])
    spectral_emb = SpectralEmbedding().fit_transform(data_dict['data'])
    embedor_euc = EmbedOR(metric='euclidean')
    embedding_euc = embedor_euc.fit_transform(data_dict['data'])
    # compute pairwise distances and calculate correlation
    from scipy.spatial.distance import pdist, squareform
    pdist_embedor = squareform(pdist(embedding, metric='euclidean'))
    pdist_umap = squareform(pdist(umap_emb, metric='euclidean'))
    pdist_tsne = squareform(pdist(tsne_emb, metric='euclidean'))
    pdist_phate = squareform(pdist(phate_emb, metric='euclidean'))
    pdist_isomap = squareform(pdist(isomap_emb, metric='euclidean'))
    pdist_spectral = squareform(pdist(spectral_emb, metric='euclidean'))
    pdist_embedor_euc = squareform(pdist(embedding_euc, metric='euclidean'))
    # compute pearson and spearman correlation
    from scipy.stats import pearsonr, spearmanr
    spearman_corr_embedor, _ = spearmanr(pdist_embedor.flatten(), gt_geodesic_distance.flatten())
    spearman_corr_umap, _ = spearmanr(pdist_umap.flatten(), gt_geodesic_distance.flatten())
    spearman_corr_tsne, _ = spearmanr(pdist_tsne.flatten(), gt_geodesic_distance.flatten())
    spearman_corr_phate, _ = spearmanr(pdist_phate.flatten(), gt_geodesic_distance.flatten())
    spearman_corr_isomap, _ = spearmanr(pdist_isomap.flatten(), gt_geodesic_distance.flatten())
    spearman_corr_spectral, _ = spearmanr(pdist_spectral.flatten(), gt_geodesic_distance.flatten())
    spearman_corr_embedor_euc, _ = spearmanr(pdist_embedor_euc.flatten(), gt_geodesic_distance.flatten())
    print()
    print(f"Spearman correlation EmbedOR: {spearman_corr_embedor}")
    print(f"Spearman correlation UMAP: {spearman_corr_umap}")
    print(f"Spearman correlation t-SNE: {spearman_corr_tsne}")
    print(f"Spearman correlation PHATE: {spearman_corr_phate}")
    print(f"Spearman correlation Isomap: {spearman_corr_isomap}")
    print(f"Spearman correlation Spectral: {spearman_corr_spectral}")
    print(f"Spearman correlation EmbedOR (euclidean): {spearman_corr_embedor_euc}")

In [11]:
# circles
%autoreload 2
from src.data.data import *
n_points = 5000
noise = 0.1
noise_thresh = None
return_dict = concentric_circles(n_points=n_points, factor=0.4, noise=noise, noise_thresh=noise_thresh)

In [12]:
geodesic_dist_experiment(return_dict)

Building nearest neighbor graph...
Computing distances...
Computing affinities...
Updating the graph attributes...
Running Stochastic Neighbor Embedding...
Calculating PHATE...
  Running PHATE on 5000 observations and 2 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 0.04 seconds.
    Calculating affinities...
    Calculated affinities in 0.01 seconds.
  Calculated graph and diffusion operator in 0.06 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 0.22 seconds.
    Calculating KMeans...
    Calculated KMeans in 1.75 seconds.
  Calculated landmark operator in 2.38 seconds.
  Calculating optimal t...
    Automatically selected t = 55
  Calculated optimal t in 1.38 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.65 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 3.01 seconds.
Calculated PHATE in 7.48 seconds.
Building nearest nei

In [13]:
noise = 1
noise_thresh = None
return_dict = swiss_roll(n_points=n_points, noise=noise, noise_thresh=noise_thresh)

geodesic_dist_experiment(return_dict)

Building nearest neighbor graph...
Computing distances...
Computing affinities...
Updating the graph attributes...
Running Stochastic Neighbor Embedding...
Calculating PHATE...
  Running PHATE on 5000 observations and 3 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 0.05 seconds.
    Calculating affinities...
    Calculated affinities in 0.01 seconds.
  Calculated graph and diffusion operator in 0.07 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 0.76 seconds.
    Calculating KMeans...
    Calculated KMeans in 2.05 seconds.
  Calculated landmark operator in 3.21 seconds.
  Calculating optimal t...
    Automatically selected t = 56
  Calculated optimal t in 1.83 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.61 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 3.04 seconds.
Calculated PHATE in 8.78 seconds.
Building nearest nei

In [14]:
noise = 0.5
noise_thresh = None
return_dict = torus(n_points=n_points, noise=noise, noise_thresh=noise_thresh, double=True)
geodesic_dist_experiment(return_dict)

Building nearest neighbor graph...
Computing distances...
Computing affinities...
Updating the graph attributes...
Running Stochastic Neighbor Embedding...
Calculating PHATE...
  Running PHATE on 5000 observations and 3 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 0.07 seconds.
    Calculating affinities...
    Calculated affinities in 0.02 seconds.
  Calculated graph and diffusion operator in 0.09 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 0.23 seconds.
    Calculating KMeans...
    Calculated KMeans in 2.23 seconds.
  Calculated landmark operator in 2.86 seconds.
  Calculating optimal t...
    Automatically selected t = 51
  Calculated optimal t in 1.74 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.68 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 3.08 seconds.
Calculated PHATE in 8.46 seconds.
Building nearest nei

In [15]:
noisy_tree, tree = gen_dla(n_dim=100, n_branch=8, sigma=4, branch_length=500)
return_dict = {'data': noisy_tree, 'noiseless_data': tree}
geodesic_dist_experiment(return_dict)

Building nearest neighbor graph...
Computing distances...
Computing affinities...
Updating the graph attributes...
Running Stochastic Neighbor Embedding...
Calculating PHATE...
  Running PHATE on 4000 observations and 100 variables.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculated KNN search in 0.51 seconds.
    Calculating affinities...
    Calculated affinities in 0.01 seconds.
  Calculated graph and diffusion operator in 0.53 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 0.22 seconds.
    Calculating KMeans...
    Calculated KMeans in 2.04 seconds.
  Calculated landmark operator in 2.65 seconds.
  Calculating optimal t...
    Automatically selected t = 5
  Calculated optimal t in 1.71 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.32 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 2.83 seconds.
Calculated PHATE in 8.05 seconds.
Building nearest ne